In [1]:
# !wget https://huggingface.co/datasets/WizardLM/WizardLM_evol_instruct_V2_196k/resolve/main/WizardLM_evol_instruct_V2_143k.json

In [2]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt4-malaysian-general-qa/resolve/main/malaysian-general-qa.jsonl
# !wget https://huggingface.co/datasets/mesolitica/chatgpt4-malaysian-general-qa/resolve/main/malaysian-general-qa-v2.jsonl

In [3]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [4]:
openai.api_type = "azure"
openai.api_base = "https://husein-ai9-aiservices1611641965.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

engine = 'gpt-35-turbo'

In [5]:
instructions = []
for f in ['malaysian-general-qa.jsonl', 'malaysian-general-qa-v2.jsonl']:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            instructions.append(l['question'])
       
instructions = instructions[::-1]
len(instructions)

25690

In [6]:
instructions[0]

'Terdapat berapa buah daerah di Negeri Sembilan dan apakah nama-nama daerah tersebut?'

In [7]:
def predict(prompt, max_tokens = 1024):
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
       
    ]
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=max_tokens,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [8]:
from depth_v2 import createConstraintsPrompt, createDeepenPrompt, createConcretizingPrompt, createReasoningPrompt
from breadth_v2 import createBreadthPrompt

In [9]:
!mkdir evol-instructs-v2
# !rm evol-instructs-v2/*

mkdir: cannot create directory ‘evol-instructs-v2’: File exists


In [10]:
def generate(instruction, index):
    filename = f'evol-instructs-v2/{index}.json'
    if os.path.exists(filename):
        return
    
    try:
        evol_prompts = []
        evol_prompts.append(createConstraintsPrompt(instruction))
        evol_prompts.append(createDeepenPrompt(instruction))
        evol_prompts.append(createConcretizingPrompt(instruction))
        evol_prompts.append(createReasoningPrompt(instruction))
        evol_prompts.append(createBreadthPrompt(instruction))

        selected_evol_prompt = random.choice(evol_prompts)
        ins = predict(selected_evol_prompt)
        answer = predict(ins + ', jawab dalam bahasa melayu', max_tokens = 2048)
        d = {"instruction":ins,"output":answer}
        with open(filename, 'w') as fopen:
            json.dump(d, fopen)
    except Exception as e:
        print(e)
        pass

In [11]:
generate(instructions[2], 2)

In [12]:
instructions[2]

'Siapakah yang merupakan ketua menteri di Putrajaya pada masa kini?'

In [13]:
from tqdm import tqdm

max_worker = 1
for i in tqdm(range((len(instructions) // 4) * 0, (len(instructions) // 4) * 1, max_worker)):
    b = instructions[i: i + max_worker]
    b = [(ins, i + no) for no, ins in enumerate(b)]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f[0], f[1]): f for f in b}

        for future in as_completed(futures):
            future.result()

100%|█████████████████████████████████████| 6422/6422 [5:20:10<00:00,  2.99s/it]
